In [11]:
from sgkit.distance.api import pairwise_distance_blocks, pairwise_distance_blockwise
import dask.array as da

from dask.distributed import Client, performance_report, get_task_stream
from dask.diagnostics import ProgressBar, ResourceProfiler
from bokeh.io import output_notebook

import numpy as np
import coiled
import fsspec, zarr

In [2]:
coiled.__version__

'0.0.34'

In [3]:
from plotly.subplots import make_subplots
import plotly.express as px

from plotly import io as pio
pio.renderers.default = "png+plotly_mimetype"
# pio.orca.config.use_xvfb = True

In [4]:
# env_name = "sgkit-blockwise-new"
env_name = "pairwise-comparison"

In [5]:
coiled.create_software_environment(
    name=env_name,
    conda="environment.yml",
)

Updating software environment...
Solving conda environment...
Conda environment solved!
Building Docker image
(this takes a few minutes)
STEP 1: FROM coiled/default:sha-9aa53a2
STEP 2: COPY environment.yml environment.yml
--> 920c3a5033d
STEP 3: RUN conda env update -n base -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate base" >> ~/.bashrc
Solving environment: ...working... done

monotonic-1.5        | 9 KB      | ########## | 100% 
nbclient-0.5.1       | 60 KB     | ########## | 100% 
partd-1.1.0          | 17 KB     | ########## | 100% 
jupyterlab_pygments- | 8 KB      | ########## | 100% 
psutil-5.8.0         | 342 KB    | ########## | 100% 
readline-8.0         | 281 KB    | ########## | 100% 
xz-5.2.5             | 343 KB    | ########## | 100% 
pickleshare-0.7.5    | 9 KB      | ########## | 100% 
sqlite-3.34.0        | 1.4 MB    | ########## | 100% 
pytz-2020.5          | 244 KB    | ########## | 100% 
requests-oauthlib-1. | 2

In [7]:
coiled.create_cluster_configuration(name=env_name, software=env_name, worker_cpu=4, worker_memory="16 GiB")

In [8]:
cluster = coiled.Cluster(configuration=f"aktech/{env_name}", n_workers=50, worker_options={
    "nthreads": 4,
    "env": {
#         "OMP_NUM_THREADS": "2",
#         "OPENBLAS_NUM_THREADS": "1",
    },
    "memory_limit": "16 GB"
})

client = Client(cluster, timeout=30)
print('Dashboard:', client.dashboard_link)

Unclosed connection
client_connection: Connection<ConnectionKey(host='cloud.coiled.io', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=6839938356316674306)>


Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found
Dashboard: http://ec2-3-133-156-161.us-east-2.compute.amazonaws.com:8787


/Users/aktech/anaconda3/envs/sgkit/lib/python3.8/site-packages/distributed/client.py:1129: VersionMismatchWarning:

Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| msgpack | 1.0.0         | 1.0.2         | 1.0.2         |
| python  | 3.8.5.final.0 | 3.8.6.final.0 | 3.8.6.final.0 |
+---------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6



In [12]:
store = fsspec.get_mapper('gs://ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1')
callset_snps = zarr.open_consolidated(store=store)
gt = callset_snps['2R/calldata/GT']

In [13]:
gt_da = da.from_zarr(gt)
x = gt_da[:, :, 1].T
x = x.rechunk((-1, 100000))
x

dask.array<rechunk-merge, shape=(1142, 24767689), dtype=int8, chunksize=(1142, 100000), chunktype=numpy.ndarray>

In [14]:
%%time
with performance_report(filename="dask-report-pairwise_distance_blockwise.html"), get_task_stream(plot='save', filename="task-stream-pairwise_distance_blockwise.html") as ts:
    out = pairwise_distance_blockwise(x)

CPU times: user 2.82 s, sys: 398 ms, total: 3.22 s
Wall time: 21min 33s


In [15]:
%%time
with performance_report(filename="dask-report-pairwise_distance_blockwise.html"), get_task_stream(plot='save', filename="task-stream-pairwise_distance_blockwise.html") as ts:
    out = pairwise_distance_blocks(x)

CPU times: user 5.13 s, sys: 390 ms, total: 5.52 s
Wall time: 20min 28s
